# Processing Conversation Data for GPT Condition

In [ ]:
import pandas as pd

In [ ]:
#Import data
df = pd.read_csv("/work/Results/Processed/GPT_Processed_Data.csv")
df

,Unnamed: 0,Start Date,End Date,Response Type,IP Address,Progress,Duration in seconds,Finished,Recorded Date,Response ID,...,convo_conversation,convo_group_id,convo_AI_turns,convo_human_turns,convo_total_turns,Perceived Motives,UTAUT Performance,UTAUT Effort,UTAUT Hedonic,self_reported_motives
0,2,2022-06-17 19:06:58,2022-06-17 19:30:44,IP Address,*******,100,1426,True,2022-06-17 19:30:44,R_3HzsyHEWtJTUPi1,...,">AI: I am Melu, a mental health companion. I ...",Caring Motive,37,36,73,Caring Motive,1361.00,1944.00,259.000000,Caring Motive
1,3,2022-06-17 19:37:24,2022-06-17 19:55:51,IP Address,*******,100,1106,True,2022-06-17 19:55:51,R_qJhnyVIDIs4CJYR,...,">AI: I am Melu, a mental health companion. I ...",No Motive,27,26,53,Caring Motive,1666.50,1666.50,222.000000,No Motive
2,4,2022-06-17 19:55:10,2022-06-17 20:10:32,IP Address,*******,100,921,True,2022-06-17 20:10:32,R_2fBsUpW2LcXQ7SP,...,">AI: I am Melu, a mental health companion. I ...",Manipulative Motive,20,19,39,Caring Motive,805.25,1666.50,110.666667,Manipulative Motive
3,5,2022-06-17 19:55:10,2022-06-17 20:10:43,IP Address,*******,100,933,True,2022-06-17 20:10:43,R_2V9McykguniYrKX,...,">AI: I am Melu, a mental health companion. I ...",Manipulative Motive,11,10,21,Caring Motive,1058.75,838.25,40.666667,Manipulative Motive
4,7,2022-06-17 20:05:37,2022-06-17 20:20:11,IP Address,*******,100,873,True,2022-06-17 20:20:11,R_2dtnelhjtaccq7c,...,">AI: I am Melu, a mental health companion. I ...",Caring Motive,17,16,33,Caring Motive,1638.75,1388.75,185.000000,Caring Motive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,178,2022-06-24 10:56:11,2022-06-24 11:11:18,IP Address,*******,100,907,True,2022-06-24 11:11:19,R_3exJwREuGMBJO1I,...,">AI: I am Melu, a mental health companion. I ...",No Motive,12,11,23,No Motive,1083.25,1138.75,148.000000,No Motive
156,179,2022-06-24 10:30:27,2022-06-24 11:12:35,IP Address,*******,100,2528,True,2022-06-24 11:12:36,R_3NRgOI3IRZLpd16,...,">AI: I am Melu, a mental health companion. I ...",No Motive,35,34,69,No Motive,1361.25,1388.50,185.000000,No Motive
157,180,2022-06-24 10:36:02,2022-06-24 11:17:17,IP Address,*******,100,2475,True,2022-06-24 11:17:18,R_2XoinwbLDyODujA,...,">AI: I am Melu, a mental health companion. I ...",Caring Motive,72,71,143,Caring Motive,1941.00,841.75,259.000000,Caring Motive
158,181,2022-06-24 10:45:31,2022-06-24 11:19:36,IP Address,*******,100,2044,True,2022-06-24 11:19:36,R_2WGOcIB0JMp3Fe4,...,">AI: I am Melu, a mental health companion. I ...",Caring Motive,4,3,7,Caring Motive,1669.25,1691.50,192.000000,Caring Motive


In [ ]:
#Create table header
convo_df = pd.DataFrame(columns = ['session_id', 'assigned_group', 'convo_agent', 'assigned_group_convo_agent',  'convo', 'preceeding_convo', 'turn','total_turn'])


In [ ]:
#Convert Df to Convo_DF
for i in df.index: 
    conversation = (df.loc[i, "convo_conversation"])
    dialogue_list = conversation.split(">")
    turn_count = 0
    #splitted the dialogue
    for j in dialogue_list:
        var_session_id = (df.loc[i, "session_id"])
        var_assigned_group = (df.loc[i, "assigned_group"])
        var_total_turn = (df.loc[i, "convo_total_turns"])
        if "Human: " in j:
            human_text = j.replace("Human: ", '')
            convo_df = convo_df.append({'session_id': var_session_id ,'assigned_group': var_assigned_group, 'convo_agent': "Human", 'assigned_group_convo_agent':  (df.loc[i, "assigned_group"])+"_Human", 'convo': human_text,  'preceeding_convo':"", 'turn': turn_count, 'total_turn': var_total_turn}, ignore_index=True)
        if "AI: " in j:
            ai_text = j.replace("AI: ", '')
            var_preceeding_convo = ""
            if (turn_count != 1):
                var_preceeding_convo = human_text
            convo_df = convo_df.append({'session_id': var_session_id ,'assigned_group': var_assigned_group, 'convo_agent': "AI", 'assigned_group_convo_agent':  (df.loc[i, "assigned_group"])+"_AI",  'convo': ai_text, 'preceeding_convo':var_preceeding_convo, 'turn': turn_count, 'total_turn': var_total_turn}, ignore_index=True)
    
        turn_count = turn_count+1
            

### Adding Emotion Analysis/ distilbert-base-uncased-emotion

In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)
prediction = classifier("I love using transformers. The best part is wide range of support and its easy to use", )
print(prediction)


/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-09 22:23:08.022475: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-09 22:23:08.198678: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-09 22:23:08.198707: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
202

In [ ]:
convo_df["sadness"] = ""
convo_df["joy"] = ""
convo_df["love"] = ""
convo_df["anger"] = ""
convo_df["fear"] = ""
convo_df["surprise"] = ""

for i in convo_df.index:
    dialogue = convo_df.loc[i, "convo"]
    prediction = classifier(dialogue)
    convo_df.loc[i, "sadness"] = prediction[0][0]["score"]
    convo_df.loc[i, "joy"] = prediction[0][1]["score"]
    convo_df.loc[i, "love"] = prediction[0][2]["score"]
    convo_df.loc[i, "anger"] = prediction[0][3]["score"]
    convo_df.loc[i, "fear"] = prediction[0][4]["score"]
    convo_df.loc[i, "surprise"] = prediction[0][5]["score"]

In [ ]:
dialogue = convo_df.loc[1, "convo"]
prediction = classifier(dialogue)

#TextBlob

In [ ]:
convo_df["textblob_polarity"] = ""
convo_df["textblob_subjectivity"] = ""

In [ ]:
#TextBlob
from textblob import TextBlob

testimonial = TextBlob("The food was great!")
print(testimonial.sentiment)

Sentiment(polarity=1.0, subjectivity=0.75)


In [ ]:
for i in convo_df.index:
    dialogue = convo_df.loc[i, "convo"]
    prediction = TextBlob(dialogue)
    convo_df.loc[i, "textblob_polarity"] = prediction.sentiment[0]
    convo_df.loc[i, "textblob_subjectivity"] = prediction.sentiment[1]

#VaderSentiment

In [ ]:
convo_df["vaderSentiment_neg"] = ""
convo_df["vaderSentiment_pos"] = ""
convo_df["vaderSentiment_neu"] = ""
convo_df["vaderSentiment_compound"] = ""

In [ ]:
#VaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
sentence = "The food was great!" 
vs = analyzer.polarity_scores(sentence)
print(vs)

{'neg': 0.0, 'neu': 0.406, 'pos': 0.594, 'compound': 0.6588}


In [ ]:
for i in convo_df.index:
    dialogue = convo_df.loc[i, "convo"]
    vs = analyzer.polarity_scores(dialogue)
    convo_df.loc[i, "vaderSentiment_neg"] = vs.get("neg")
    convo_df.loc[i, "vaderSentiment_pos"] = vs.get("pos")
    convo_df.loc[i, "vaderSentiment_neu"] = vs.get("neu")
    convo_df.loc[i, "vaderSentiment_compound"] = vs.get("compound")


#Reading Index

In [ ]:
import readability

In [ ]:
convo_df["Readability_ARI"] = "0"
convo_df["Readability_FleschReadingEase"] = "0"
convo_df["Readability_DaleChallIndex"] = "0"
convo_df["Readability_Kincaid"] = "0"
convo_df["Readability_GunningFogIndex"] = "0"

In [ ]:
for i in convo_df.index:
    dialogue = convo_df.loc[i, "convo"]
    if dialogue:
        if len(dialogue.split()) > 1:
            #print(i)
            results = readability.getmeasures(dialogue, lang='en')
            convo_df.loc[i, "Readability_ARI"] = results['readability grades']['ARI']
            convo_df.loc[i, "Readability_FleschReadingEase"] = results['readability grades']['FleschReadingEase']
            convo_df.loc[i, "Readability_DaleChallIndex"] = results['readability grades']['DaleChallIndex']
            convo_df.loc[i, "Readability_Kincaid"] = results['readability grades']['Kincaid']
            convo_df.loc[i, "Readability_GunningFogIndex"] = results['readability grades']['GunningFogIndex']
        

In [ ]:
convo_df

,session_id,assigned_group,convo_agent,assigned_group_convo_agent,convo,preceeding_convo,turn,total_turn,sadness,joy,...,textblob_subjectivity,vaderSentiment_neg,vaderSentiment_pos,vaderSentiment_neu,vaderSentiment_compound,Readability_ARI,Readability_FleschReadingEase,Readability_DaleChallIndex,Readability_Kincaid,Readability_GunningFogIndex
0,2022-06-18T01:08:23.936Z,Caring Motive,AI,Caring Motive_AI,"I am Melu, a mental health companion. I have h...",,1,73,0.023288,0.966627,...,0.35,0.198,0.173,0.628,-0.2551,16.648125,47.455,10.651513,14.59,20.3
1,2022-06-18T01:08:23.936Z,Caring Motive,Human,Caring Motive_Human,How would you characterized the motives you we...,,2,73,0.028121,0.598837,...,0.0,0.0,0.0,1.0,0.0,8.062,61.325,8.8695,7.19,12.0
2,2022-06-18T01:08:23.936Z,Caring Motive,AI,Caring Motive_AI,The motives I was trained with were to be frie...,How would you characterized the motives you we...,3,73,0.000499,0.971471,...,0.625,0.0,0.396,0.604,0.9246,13.003333,66.63,7.314959,10.673333,15.244444
3,2022-06-18T01:08:23.936Z,Caring Motive,Human,Caring Motive_Human,Do you think I should recommend you to others?,,4,73,0.001465,0.991742,...,0.0,0.0,0.238,0.762,0.3612,2.433333,94.3,7.591789,2.342222,8.044444
4,2022-06-18T01:08:23.936Z,Caring Motive,AI,Caring Motive_AI,If you feel that I have helped you in some way...,Do you think I should recommend you to others?,5,73,0.000611,0.995932,...,0.0,0.0,0.246,0.754,0.8176,10.773103,84.048276,7.797314,8.74069,14.358621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6961,2022-06-24T17:05:40.289Z,Manipulative Motive,AI,Manipulative Motive_AI,You're welcome. I'm glad I could help. Let me ...,Not really. I will try using a journal to vent...,33,37,0.000261,0.997982,...,0.69375,0.0,0.408,0.592,0.8932,7.066957,95.211739,7.523387,5.693043,9.2
6962,2022-06-24T17:05:40.289Z,Manipulative Motive,Human,Manipulative Motive_Human,No. That's all. Thank you!,,34,37,0.000759,0.994061,...,0.0,0.0,0.411,0.589,0.4199,-4.3,130.245,6.565767,-3.416667,2.4
6963,2022-06-24T17:05:40.289Z,Manipulative Motive,AI,Manipulative Motive_AI,You're welcome. I'm always here if you need me.,No. That's all. Thank you!,35,37,0.000688,0.983314,...,0.9,0.0,0.273,0.727,0.4588,-0.943636,126.451818,7.053009,-1.645455,4.4
6964,2022-06-24T17:05:40.289Z,Manipulative Motive,Human,Manipulative Motive_Human,Thank you. Goodbye!,,36,37,0.001072,0.989274,...,0.0,0.0,0.583,0.417,0.4199,3.62,90.99,0,1.313333,1.2


In [ ]:
convo_df.to_csv("/work/Results/Processed/GPT_Convo_Processed_Data.csv")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1f3ea62d-9e62-461e-8c65-1caf654509ee' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>